In [1]:
import json
import requests
import pandas as pd

In [2]:
api_records = []

In [3]:
page = 1

while 1:
    url = f"http://localhost:8000/recommendation_aggregate/recommendation-aggregate-data?page={page}&page_size=100"
    payload = {}
    headers = {
      'accept': 'application/json'
    }
    
    response = requests.request("GET", url, headers=headers, data=payload)
    result = response.json()

    if len(result) == 0:
        break

    api_records.extend(result)
    page += 1    

In [4]:
len(api_records)

824

In [5]:
comment_list = []
for element in api_records:
    if len(element["restaurant_comments"]) == 0:
        continue
        
    for comment in element["restaurant_comments"]:
        try:
            if isinstance(comment, str):
                comment = json.loads(comment.replace('\\"', "").replace('\"Yola çıktı"', '\\"Yola çıktı\\"'))  
            comment_list.append(comment)
        except Exception as e:
            print(e)
            print(comment, type(comment))
        

Expecting value: line 1 column 157 (char 156)
{"id":"a8a6ffd27fa9b2da2338770c863554b37ffbf983729d253533c7307a95af2f12","restaurant_id":"5fe246f923ff893055a771c8","provider":"GETIR","rating":1,"comment":"\"Yola çıktı" yazmasına rağmen tam 2 saat bekledik ve siparişimiz gelmedi !!!","comment_id":"668d49cb16ff6ff6ef4b92a3","replies":[],"like_count":0,"created_at":"","updated_at":"","version":1733688239} <class 'str'>
Expecting ',' delimiter: line 1 column 269 (char 268)
{"id":"2a5cc05ecb9fcc25627be31c7456c7387535235415afdccf20c3ab71dddee9a4","restaurant_id":"60097d7df69aa47d89474a17","provider":"GETIR","rating":1,"comment":"sipariş için zile basmayın seçeneğini işaretledim. üstüne birde sipariş notu ekledim \"zile kesinlikle basmayın" diye. işine saygısı olan insanlar bu gibi şeylere dikkat eder. getir uygulamasının da siparişi gönderen işletmenin de işine saygısı yok benim gözümde. siparişlerinizi bu kıstasa göre verirsiniz. umarım yaptığım yorum sipariş verecek kişiler için aydınlatıcı 

In [6]:
restaurant_list = []
for element in api_records:
    del element["restaurant_comments"]
    del element["menu_items"]
    
    restaurant_list.append(element)

In [7]:
comments = pd.DataFrame(comment_list)
restaurants = pd.DataFrame(restaurant_list)

In [8]:
restaurant_map = restaurants.set_index("restaurant_id").to_dict(orient="index")

def enrich_comment(row):
    restaurant = restaurant_map.get(row["restaurant_id"], {})
    return (
        f"passage: Comment: {row['comment']}. "
        f"Rating: {row['rating']}/5. "
        f"Restaurant: {restaurant.get('name', '')}, "
        f"City: {restaurant.get('city', '')}. "
        f"Restaurant Rating: {restaurant.get('rating', 'N/A')}/5."
    )

comments["enriched"] = comments.apply(enrich_comment, axis=1)


In [9]:
comments["enriched"]

0       passage: Comment: Tavuk aşırı kuruydu. Rating:...
1       passage: Comment: Kolamız eksik getirildi geti...
2       passage: Comment: Her zamanki gibi zamanında g...
3       passage: Comment: Porsiyonlar dolu dolu ve sıc...
4       passage: Comment: sürekli yediğimiz yer gayet ...
                              ...                        
1260    passage: Comment: başak kır pidesi.. dükkanın ...
1261    passage: Comment: benim siparişim gelmedi ..ba...
1262    passage: Comment: tavuklu pide biraz acı bilhi...
1263    passage: Comment: tek kelimeyle muhteşem. Rati...
1264    passage: Comment: Hem paramı aldı bu işletme h...
Name: enriched, Length: 1265, dtype: object

In [10]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/multilingual-e5-base")

embeddings = model.encode(comments["enriched"].tolist(), normalize_embeddings=True)

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def semantic_search(query, embeddings, top_k=3):
    query_embedding = model.encode(f"query: {query}", normalize_embeddings=True)
    similarities = cosine_similarity([query_embedding], embeddings)[0]
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return [(comments.iloc[i], similarities[i]) for i in top_indices]


In [27]:
results = semantic_search("miğdem bulandı", embeddings, top_k=15)
print('\n'.join([res[0]["comment"] for res in results]))

Pul biber koymayın dediğim halde koyulmuştu. Aşırı geç geldi. Mağdur oldum. Ekmeği de bayattı
Çilekli magnolia sipariş etmiştim ama muzlu çıktı❌Ürün elinizde yoksa müşteriyi aramalı ya da siparişi iptal etmelisiniz😕Ayrıca muzları kararmış yumuşamıştı, tatlının tadı güzel gelmedi bana 👎🏻
keske catal da gonderselerdi daha rahat yiyebilirdim, muhtemelen unuttular zaten cok sorun degil ama unutulmasa daha iyiydi. onun disinda lezzeti gayet yerindeydi
kurye pos cihazını unuttu, uğrayacağını söyledikten sonra saatlerce beni bekletti
LEZZET GÜZEL FAKAT MİKTAR O KADAR AZ Kİ. TADIMLIK SANKİ
soğan istemiştim gelmedi
Dürüm soğuk geldi üstelik baharatlar çok yoğundu.
gerçekten süperdi elinize sağlık. ciğer dürüm menüyü asla bozmayın. en kısa zamanda yine sipariş vericem
mercimek soledik ezo gelin geldi yoksa arayip bilgi verilmesi gerek ya
Kuryeniz kabaydı. Saygısızca konuştu
Kuzu ciğer yerine dana ciğer geldi , tadı berbattı , yemesi işkence gibiydi
1.5 saatte geldi ve buz gibiydi .!
Posrsiyon bü

In [ ]:
cases = [
    "servis kısmen iyiydi",
    "kurye geç geldi",
    "miğdem bulandı"
]